# Lab 11 v2: Deploy a Foundry A365 Agent with Azure Developer CLI

Deploy a **Foundry A365 agent** using the official Microsoft sample and **Azure Developer CLI (azd)**.

> ⚠️ **Frontier Preview Access Required**
> 
> You must be enrolled in the [Frontier preview program](https://adoption.microsoft.com/en-us/copilot/frontier-program/) to publish a Foundry agent to Microsoft Agent 365.

## What This Deploys

This sample orchestrates **six key components**:

| Component | Description |
|-----------|-------------|
| **1. Foundry Project** | Creates a Foundry project with Container Registry access |
| **2. Application** | Stable endpoint and identity for the agent |
| **3. Azure Bot Service** | Relay between M365 and the Foundry application |
| **4. Hosted Agent** | Docker container built and registered with Foundry |
| **5. Agent Deployment** | Deploys agent to serve application requests |
| **6. M365 Publishing** | Publishes as a hireable digital worker |

## Prerequisites

| Requirement | Description |
|------------|-------------|
| [Azure Developer CLI](https://learn.microsoft.com/azure/developer/azure-developer-cli/install-azd) | Infrastructure deployment tool |
| [Docker](https://docker.com) | Container runtime |
| [.NET 9.0 SDK](https://dotnet.microsoft.com/download) | Development framework |
| **Owner** role | On the Azure subscription |
| **Azure AI User** or **Cognitive Services User** | At subscription or resource group level |
| **Tenant Admin** | For organization-wide configuration |

---
## Step 1: Clone the Official Sample

Clone the FoundryA365 sample from the Microsoft Foundry samples repository.

In [ ]:
import os
import subprocess

# Define paths
sample_dir = "foundry-a365-sample"
repo_url = "https://github.com/microsoft-foundry/foundry-samples.git"
sample_path = "samples/csharp/FoundryA365"

# Clean up if exists
if os.path.exists(sample_dir):
    print(f"🧹 Removing existing {sample_dir}...")
    !rm -rf {sample_dir}

# Sparse checkout just the FoundryA365 sample
print(f"📥 Cloning FoundryA365 sample...")
!git clone --filter=blob:none --sparse {repo_url} {sample_dir}

os.chdir(sample_dir)
!git sparse-checkout set {sample_path}

# Move contents up and clean
os.chdir("..")
!mv {sample_dir}/{sample_path}/* {sample_dir}/
!rm -rf {sample_dir}/samples

print(f"\n✅ Sample cloned to: {sample_dir}/")
!ls -la {sample_dir}/

---
## Step 2: Install Prerequisites

Install .NET 9.0 SDK and Azure Developer CLI.

In [ ]:
# Install .NET SDK 9.0
print("📦 Installing .NET 9.0 SDK...")
!wget -q https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --channel 9.0

# Add to PATH for current session
import os
os.environ["PATH"] = f"{os.environ['HOME']}/.dotnet:{os.environ['PATH']}"
os.environ["DOTNET_ROOT"] = f"{os.environ['HOME']}/.dotnet"

# Verify installation
!$HOME/.dotnet/dotnet --version
print("✅ .NET SDK installed")

In [ ]:
# Install Azure Developer CLI (azd)
print("📦 Installing Azure Developer CLI...")
!curl -fsSL https://aka.ms/install-azd.sh | bash

# Add to PATH
os.environ["PATH"] = f"{os.environ['HOME']}/.azd/bin:{os.environ['PATH']}"

# Verify installation
!azd version
print("✅ Azure Developer CLI installed")

In [ ]:
# Verify Docker is running
print("🐳 Checking Docker...")
!docker --version
!docker info --format '{{.ServerVersion}}' 2>/dev/null || echo "⚠️ Docker daemon not running - please start Docker Desktop"

---
## Step 3: Authenticate with Azure

Login to Azure CLI and Azure Developer CLI with the required scopes.

In [ ]:
# Login to Azure CLI
print("🔐 Logging into Azure CLI...")
!az login --use-device-code

In [ ]:
# Login with AI scope (required for Foundry operations)
print("🔐 Acquiring AI scope token...")
!az login --scope https://ai.azure.com/.default --use-device-code

In [ ]:
# Login with Graph scope (required for publishing)
print("🔐 Acquiring Graph scope token...")
!az login --scope https://graph.microsoft.com//.default --use-device-code

In [ ]:
# Login to Azure Developer CLI
print("🔐 Logging into Azure Developer CLI...")
!azd auth login --use-device-code

---
## Step 4: (Optional) Customize Your Agent

Before deploying, you can customize:

| File | Purpose |
|------|--------|
| `src/hello_world_a365_agent/AgentLogic/AgentInstructions.cs` | Agent instructions/persona |
| `src/hello_world_a365_agent/ToolingManifest.json` | MCP tools configuration |

### Default Agent Instructions

In [ ]:
# View current agent instructions
instructions_file = "foundry-a365-sample/src/hello_world_a365_agent/AgentLogic/AgentInstructions.cs"
print(f"📄 Current agent instructions ({instructions_file}):\n")
!cat {instructions_file}

In [ ]:
# View current MCP tools configuration
tooling_file = "foundry-a365-sample/src/hello_world_a365_agent/ToolingManifest.json"
print(f"📄 Current MCP tools ({tooling_file}):\n")
!cat {tooling_file}

### Available MCP Servers

| Server | Capabilities |
|--------|-------------|
| `mcp_MailServer` | Send, read, search emails |
| `mcp_CalendarServer` | Create, update calendar events |
| `mcp_TeamsServer` | Post messages, manage chats |
| `mcp_OneDriveServer` | Access OneDrive files |
| `mcp_WordServer` | Create and edit Word documents |
| `mcp_ExcelServer` | Work with Excel spreadsheets |
| `mcp_PowerPointServer` | Create presentations |

---
## Step 5: Deploy with Azure Developer CLI

> **Important:** Hosted agents are only available in the **North Central US** region. All resources will be created in this region.

In [ ]:
# Change to sample directory
os.chdir("foundry-a365-sample")
print(f"📂 Working directory: {os.getcwd()}")
!ls -la

In [ ]:
# Initialize azd environment (interactive)
# This will prompt for:
#   - Environment name (e.g., "foundry-a365-dev")
#   - Azure subscription
#   - Location (use northcentralus)

print("🚀 Initializing Azure Developer CLI environment...")
print("⚠️  When prompted for location, select: northcentralus")
!azd init

In [ ]:
# Deploy everything!
# This will:
#   1. Create Foundry project with Container Registry
#   2. Create application with stable endpoint
#   3. Set up Azure Bot Service
#   4. Build and push Docker image
#   5. Create and start the hosted agent
#   6. Publish to Microsoft 365

print("🚀 Deploying Foundry A365 Agent...")
print("⏱️  This may take 10-15 minutes...")
!azd provision --verbose

In [ ]:
# Get deployed resource values
print("📋 Deployment outputs:")
!azd env get-values

---
## Step 6: Approve Your Agent in M365 Admin Center

After deployment, your agent needs admin approval:

1. Navigate to the [Microsoft 365 admin center](https://admin.cloud.microsoft/?#/agents/all/requested)
2. Under **Requests**, locate your agent
3. Click **Approve request and activate**

![Agent Approval](https://raw.githubusercontent.com/microsoft-foundry/foundry-samples/main/samples/csharp/FoundryA365/image-1.png)

In [ ]:
# Open M365 Admin Center (run in terminal or copy URL)
admin_url = "https://admin.cloud.microsoft/?#/agents/all/requested"
print(f"🔗 Open in browser: {admin_url}")
print("\n📝 Steps:")
print("   1. Find your agent under 'Requests'")
print("   2. Click 'Approve request and activate'")

---
## Step 7: Configure Teams Integration

Set up your agent in the Teams Developer Portal.

In [ ]:
# Get the Blueprint ID from deployment
import subprocess
result = subprocess.run(["azd", "env", "get-values"], capture_output=True, text=True)
env_values = dict(line.split('=', 1) for line in result.stdout.strip().split('\n') if '=' in line)

blueprint_id = env_values.get('AGENT_IDENTITY_BLUEPRINT_ID', '').strip('"')
print(f"🔑 Blueprint ID: {blueprint_id}")
print(f"\n🔗 Teams Developer Portal: https://dev.teams.microsoft.com/tools/agent-blueprint")
print(f"\n📝 Steps:")
print(f"   1. Open Teams Developer Portal")
print(f"   2. Find your agent blueprint (may need to manually enter ID in URL)")
print(f"   3. Go to 'Configuration'")
print(f"   4. Add Bot ID: {blueprint_id}")

---
## Step 8: Create Agent Instance in Teams

Hire your agent in Microsoft Teams:

1. Open **Microsoft Teams**
2. Navigate to **Apps** → **Agents for your team**
3. Find your agent and create an instance
4. Start chatting!

![Create Agent Instance](https://raw.githubusercontent.com/microsoft-foundry/foundry-samples/main/samples/csharp/FoundryA365/image-4.png)

---
## Cleanup

Remove all deployed resources when done.

In [ ]:
# ⚠️ Uncomment to delete all resources
# print("🧹 Cleaning up resources...")
# !azd down --force --purge

---
## Troubleshooting

### Common Issues

| Issue | Solution |
|-------|----------|
| `Docker not running` | Start Docker Desktop |
| `Region not available` | Use `northcentralus` - hosted agents only available there |
| `Permission denied` | Ensure Owner role on subscription |
| `Blueprint not found in Teams` | Manually enter Blueprint ID in Teams portal URL |
| `Agent not visible after approval` | Wait 5-10 minutes for propagation |

### Useful Commands

In [ ]:
# Check deployment status
!azd env get-values

# View container logs (if deployed)
# !az container logs --resource-group <RG> --name <CONTAINER>

---
## Additional Resources

- [Foundry Container Agents Documentation](https://github.com/microsoft/container_agents_docs)
- [Azure Developer CLI Documentation](https://learn.microsoft.com/azure/developer/azure-developer-cli/)
- [Microsoft Agent 365 Documentation](https://learn.microsoft.com/en-us/microsoft-agent-365/)
- [Frontier Preview Program](https://adoption.microsoft.com/en-us/copilot/frontier-program/)